In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

# Data loading

In [3]:
df = pd.read_csv("sudoku.csv")

In [4]:
df.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


# Data processing

In [5]:
number_of_samples = 100000
unsolved = df['quizzes'].iloc[:number_of_samples].values
solved = df['solutions'].iloc[:number_of_samples].values

In [6]:
X = []
y = []

In [7]:
for sudoku in unsolved:
    np_solved = np.array([int(x) for x in sudoku]).reshape(9,9,1)
    X.append(np_solved)
    
for sudoku in solved:
    np_unsolved = np.array([int(y) for y in sudoku]).reshape(9,9,1)
    y.append(np_unsolved)

In [8]:
np_X = np.array(X)
np_y = np.array(y)

In [9]:
type(np_X)

numpy.ndarray

## Train test split

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
#default test size = 75%
X_train, X_test, y_train, y_test = train_test_split(np_X, np_y)

In [12]:
#one hot encoding
X_train = to_categorical(X_train).astype('float32')
#X_test = to_categorical(X_test).astype('float32')
X_test = X_test.astype('float32')
#-1 because we don't want to predict zeros
y_train = to_categorical(y_train-1).astype('float32')
#y_test = to_categorical(y_test-1).astype('float32')
y_test = y_test.astype('float32')

In [13]:
X_test.shape

(25000, 9, 9, 1)

# Training

In [14]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Reshape, BatchNormalization, Activation, Input,Dropout

In [15]:
#rows,cols, possible numbers in cell (0-9)
input_shape = (9,9,10)
batch_size = 128

In [16]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=input_shape))
#model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.4))
model.add(Flatten())

grid = Input(shape=input_shape)
features = model(grid)

top_model = [
    Dense(9, activation='softmax')(features)
    for i in range(81)
]
sudoku_solver = Model(grid, top_model)
sudoku_solver.compile (optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
features.shape

TensorShape([None, 5184])

In [18]:
sudoku_solver.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9, 9, 10)]   0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 5184)         4864        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 9)            46665       sequential[1][0]                 
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 9)            46665       sequential[1][0]                 
______________________________________________________________________________________________

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
early_stop = EarlyStopping(monitor='val_loss',patience=2, restore_best_weights=True)

In [21]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [21]:
with tf.device('/GPU:0'):
    sudoku_solver.fit(X_train, [y_train[:, i, j, :] for i in range(9) for j in range(9)], epochs=5, batch_size = batch_size)

Train on 75000 samples
Epoch 1/5


53760/75000 [====================>.........] - ETA: 27s - loss: 74.6077 - dense_2_loss: 0.9245 - dense_3_loss: 0.9309 - dense_4_loss: 0.9299 - dense_5_loss: 0.9126 - dense_6_loss: 0.9225 - dense_7_loss: 0.9174 - dense_8_loss: 0.9252 - dense_9_loss: 0.9218 - dense_10_loss: 0.9160 - dense_11_loss: 0.9170 - dense_12_loss: 0.9165 - dense_13_loss: 0.9180 - dense_14_loss: 0.9234 - dense_15_loss: 0.9272 - dense_16_loss: 0.9275 - dense_17_loss: 0.9141 - dense_18_loss: 0.9163 - dense_19_loss: 0.9181 - dense_20_loss: 0.9128 - dense_21_loss: 0.9086 - dense_22_loss: 0.9111 - dense_23_loss: 0.9178 - dense_24_loss: 0.9178 - dense_25_loss: 0.9165 - dense_26_loss: 0.9271 - dense_27_loss: 0.9338 - dense_28_loss: 0.9246 - dense_29_loss: 0.9279 - dense_30_loss: 0.9201 - dense_31_loss: 0.9292 - dense_32_loss: 0.9230 - dense_33_loss: 0.9266 - dense_34_loss: 0.9210 - dense_35_loss: 0.9134 - dense_36_loss: 0.9155 - dense_37_loss: 0.9182 - dense_38_loss: 0.9145 - dense_39_loss: 0.9097 - dense_40_loss: 0.9190 

53888/75000 [====================>.........] - ETA: 27s - loss: 74.5289 - dense_2_loss: 0.9239 - dense_3_loss: 0.9296 - dense_4_loss: 0.9290 - dense_5_loss: 0.9119 - dense_6_loss: 0.9214 - dense_7_loss: 0.9163 - dense_8_loss: 0.9242 - dense_9_loss: 0.9208 - dense_10_loss: 0.9153 - dense_11_loss: 0.9159 - dense_12_loss: 0.9158 - dense_13_loss: 0.9170 - dense_14_loss: 0.9226 - dense_15_loss: 0.9262 - dense_16_loss: 0.9263 - dense_17_loss: 0.9131 - dense_18_loss: 0.9154 - dense_19_loss: 0.9174 - dense_20_loss: 0.9119 - dense_21_loss: 0.9079 - dense_22_loss: 0.9100 - dense_23_loss: 0.9169 - dense_24_loss: 0.9166 - dense_25_loss: 0.9157 - dense_26_loss: 0.9261 - dense_27_loss: 0.9329 - dense_28_loss: 0.9237 - dense_29_loss: 0.9267 - dense_30_loss: 0.9189 - dense_31_loss: 0.9283 - dense_32_loss: 0.9221 - dense_33_loss: 0.9258 - dense_34_loss: 0.9198 - dense_35_loss: 0.9122 - dense_36_loss: 0.9146 - dense_37_loss: 0.9172 - dense_38_loss: 0.9141 - dense_39_loss: 0.9091 - dense_40_loss: 0.9178 

54016/75000 [====================>.........] - ETA: 27s - loss: 74.4489 - dense_2_loss: 0.9229 - dense_3_loss: 0.9283 - dense_4_loss: 0.9281 - dense_5_loss: 0.9106 - dense_6_loss: 0.9206 - dense_7_loss: 0.9155 - dense_8_loss: 0.9236 - dense_9_loss: 0.9199 - dense_10_loss: 0.9139 - dense_11_loss: 0.9150 - dense_12_loss: 0.9147 - dense_13_loss: 0.9161 - dense_14_loss: 0.9217 - dense_15_loss: 0.9253 - dense_16_loss: 0.9252 - dense_17_loss: 0.9120 - dense_18_loss: 0.9146 - dense_19_loss: 0.9163 - dense_20_loss: 0.9108 - dense_21_loss: 0.9069 - dense_22_loss: 0.9091 - dense_23_loss: 0.9161 - dense_24_loss: 0.9156 - dense_25_loss: 0.9144 - dense_26_loss: 0.9252 - dense_27_loss: 0.9321 - dense_28_loss: 0.9227 - dense_29_loss: 0.9258 - dense_30_loss: 0.9182 - dense_31_loss: 0.9270 - dense_32_loss: 0.9211 - dense_33_loss: 0.9250 - dense_34_loss: 0.9189 - dense_35_loss: 0.9111 - dense_36_loss: 0.9138 - dense_37_loss: 0.9161 - dense_38_loss: 0.9133 - dense_39_loss: 0.9080 - dense_40_loss: 0.9165 

54144/75000 [====================>.........] - ETA: 26s - loss: 74.3712 - dense_2_loss: 0.9218 - dense_3_loss: 0.9274 - dense_4_loss: 0.9268 - dense_5_loss: 0.9095 - dense_6_loss: 0.9198 - dense_7_loss: 0.9144 - dense_8_loss: 0.9227 - dense_9_loss: 0.9190 - dense_10_loss: 0.9133 - dense_11_loss: 0.9142 - dense_12_loss: 0.9136 - dense_13_loss: 0.9151 - dense_14_loss: 0.9205 - dense_15_loss: 0.9246 - dense_16_loss: 0.9241 - dense_17_loss: 0.9112 - dense_18_loss: 0.9134 - dense_19_loss: 0.9153 - dense_20_loss: 0.9098 - dense_21_loss: 0.9057 - dense_22_loss: 0.9081 - dense_23_loss: 0.9151 - dense_24_loss: 0.9146 - dense_25_loss: 0.9134 - dense_26_loss: 0.9242 - dense_27_loss: 0.9310 - dense_28_loss: 0.9220 - dense_29_loss: 0.9250 - dense_30_loss: 0.9171 - dense_31_loss: 0.9259 - dense_32_loss: 0.9198 - dense_33_loss: 0.9241 - dense_34_loss: 0.9179 - dense_35_loss: 0.9099 - dense_36_loss: 0.9128 - dense_37_loss: 0.9150 - dense_38_loss: 0.9125 - dense_39_loss: 0.9068 - dense_40_loss: 0.9154 

...

12544/75000 [====>.........................] - ETA: 44s - loss: 29.0471 - dense_2_loss: 0.3641 - dense_3_loss: 0.3630 - dense_4_loss: 0.3556 - dense_5_loss: 0.3468 - dense_6_loss: 0.3508 - dense_7_loss: 0.3498 - dense_8_loss: 0.3596 - dense_9_loss: 0.3720 - dense_10_loss: 0.3467 - dense_11_loss: 0.3476 - dense_12_loss: 0.3613 - dense_13_loss: 0.3561 - dense_14_loss: 0.3581 - dense_15_loss: 0.3654 - dense_16_loss: 0.3575 - dense_17_loss: 0.3685 - dense_18_loss: 0.3620 - dense_19_loss: 0.3579 - dense_20_loss: 0.3569 - dense_21_loss: 0.3360 - dense_22_loss: 0.3596 - dense_23_loss: 0.3553 - dense_24_loss: 0.3534 - dense_25_loss: 0.3579 - dense_26_loss: 0.3671 - dense_27_loss: 0.3641 - dense_28_loss: 0.3654 - dense_29_loss: 0.3705 - dense_30_loss: 0.3577 - dense_31_loss: 0.3590 - dense_32_loss: 0.3519 - dense_33_loss: 0.3578 - dense_34_loss: 0.3595 - dense_35_loss: 0.3534 - dense_36_loss: 0.3611 - dense_37_loss: 0.3542 - dense_38_loss: 0.3501 - dense_39_loss: 0.3542 - dense_40_loss: 0.3535 

13952/75000 [====>.........................] - ETA: 43s - loss: 29.1684 - dense_2_loss: 0.3687 - dense_3_loss: 0.3619 - dense_4_loss: 0.3571 - dense_5_loss: 0.3468 - dense_6_loss: 0.3514 - dense_7_loss: 0.3509 - dense_8_loss: 0.3611 - dense_9_loss: 0.3718 - dense_10_loss: 0.3511 - dense_11_loss: 0.3493 - dense_12_loss: 0.3645 - dense_13_loss: 0.3603 - dense_14_loss: 0.3611 - dense_15_loss: 0.3677 - dense_16_loss: 0.3596 - dense_17_loss: 0.3687 - dense_18_loss: 0.3629 - dense_19_loss: 0.3585 - dense_20_loss: 0.3581 - dense_21_loss: 0.3392 - dense_22_loss: 0.3611 - dense_23_loss: 0.3575 - dense_24_loss: 0.3519 - dense_25_loss: 0.3613 - dense_26_loss: 0.3713 - dense_27_loss: 0.3641 - dense_28_loss: 0.3645 - dense_29_loss: 0.3681 - dense_30_loss: 0.3584 - dense_31_loss: 0.3648 - dense_32_loss: 0.3523 - dense_33_loss: 0.3585 - dense_34_loss: 0.3625 - dense_35_loss: 0.3547 - dense_36_loss: 0.3631 - dense_37_loss: 0.3554 - dense_38_loss: 0.3518 - dense_39_loss: 0.3581 - dense_40_loss: 0.3563 

14080/75000 [====>.........................] - ETA: 43s - loss: 29.1832 - dense_2_loss: 0.3677 - dense_3_loss: 0.3613 - dense_4_loss: 0.3578 - dense_5_loss: 0.3476 - dense_6_loss: 0.3518 - dense_7_loss: 0.3522 - dense_8_loss: 0.3609 - dense_9_loss: 0.3726 - dense_10_loss: 0.3498 - dense_11_loss: 0.3498 - dense_12_loss: 0.3641 - dense_13_loss: 0.3602 - dense_14_loss: 0.3616 - dense_15_loss: 0.3687 - dense_16_loss: 0.3606 - dense_17_loss: 0.3682 - dense_18_loss: 0.3631 - dense_19_loss: 0.3597 - dense_20_loss: 0.3596 - dense_21_loss: 0.3392 - dense_22_loss: 0.3627 - dense_23_loss: 0.3568 - dense_24_loss: 0.3524 - dense_25_loss: 0.3617 - dense_26_loss: 0.3716 - dense_27_loss: 0.3636 - dense_28_loss: 0.3647 - dense_29_loss: 0.3684 - dense_30_loss: 0.3574 - dense_31_loss: 0.3643 - dense_32_loss: 0.3524 - dense_33_loss: 0.3595 - dense_34_loss: 0.3636 - dense_35_loss: 0.3551 - dense_36_loss: 0.3628 - dense_37_loss: 0.3553 - dense_38_loss: 0.3524 - dense_39_loss: 0.3580 - dense_40_loss: 0.3549 

39168/75000 [==============>...............] - ETA: 25s - loss: 30.6717 - dense_2_loss: 0.3849 - dense_3_loss: 0.3802 - dense_4_loss: 0.3830 - dense_5_loss: 0.3718 - dense_6_loss: 0.3764 - dense_7_loss: 0.3780 - dense_8_loss: 0.3769 - dense_9_loss: 0.3840 - dense_10_loss: 0.3752 - dense_11_loss: 0.3694 - dense_12_loss: 0.3763 - dense_13_loss: 0.3724 - dense_14_loss: 0.3818 - dense_15_loss: 0.3852 - dense_16_loss: 0.3770 - dense_17_loss: 0.3774 - dense_18_loss: 0.3745 - dense_19_loss: 0.3761 - dense_20_loss: 0.3758 - dense_21_loss: 0.3689 - dense_22_loss: 0.3763 - dense_23_loss: 0.3741 - dense_24_loss: 0.3742 - dense_25_loss: 0.3784 - dense_26_loss: 0.3828 - dense_27_loss: 0.3857 - dense_28_loss: 0.3798 - dense_29_loss: 0.3879 - dense_30_loss: 0.3789 - dense_31_loss: 0.3822 - dense_32_loss: 0.3735 - dense_33_loss: 0.3768 - dense_34_loss: 0.3789 - dense_35_loss: 0.3727 - dense_36_loss: 0.3793 - dense_37_loss: 0.3809 - dense_38_loss: 0.3709 - dense_39_loss: 0.3739 - dense_40_loss: 0.3715 

75000/75000 [==============================] - 91s 1ms/sample - loss: 31.4417 - dense_2_loss: 0.3902 - dense_3_loss: 0.3926 - dense_4_loss: 0.3906 - dense_5_loss: 0.3843 - dense_6_loss: 0.3868 - dense_7_loss: 0.3894 - dense_8_loss: 0.3875 - dense_9_loss: 0.3876 - dense_10_loss: 0.3855 - dense_11_loss: 0.3830 - dense_12_loss: 0.3835 - dense_13_loss: 0.3844 - dense_14_loss: 0.3912 - dense_15_loss: 0.3926 - dense_16_loss: 0.3906 - dense_17_loss: 0.3847 - dense_18_loss: 0.3830 - dense_19_loss: 0.3860 - dense_20_loss: 0.3857 - dense_21_loss: 0.3807 - dense_22_loss: 0.3846 - dense_23_loss: 0.3835 - dense_24_loss: 0.3810 - dense_25_loss: 0.3831 - dense_26_loss: 0.3919 - dense_27_loss: 0.3925 - dense_28_loss: 0.3907 - dense_29_loss: 0.3955 - dense_30_loss: 0.3886 - dense_31_loss: 0.3930 - dense_32_loss: 0.3858 - dense_33_loss: 0.3898 - dense_34_loss: 0.3864 - dense_35_loss: 0.3836 - dense_36_loss: 0.3869 - dense_37_loss: 0.3884 - dense_38_loss: 0.3839 - dense_39_loss: 0.3859 - dense_40_loss: 0

In [32]:
from tensorflow.keras.models import load_model

In [23]:
#sudoku_solver.save("model2.h5")

In [33]:
model2 = load_model("model2.h5")

In [34]:
model = load_model("model3.h5")

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [45]:
acc, solv = solve_batch(X_test[:1000], y_test[:1000], model)

Solved sudoku number: (0,)
Solved sudoku number: (1,)
Solved sudoku number: (2,)
Solved sudoku number: (3,)
Solved sudoku number: (4,)
Solved sudoku number: (5,)
Solved sudoku number: (6,)
Solved sudoku number: (7,)
Solved sudoku number: (8,)
Solved sudoku number: (9,)
Solved sudoku number: (10,)
Solved sudoku number: (11,)
Solved sudoku number: (12,)
Solved sudoku number: (13,)
Solved sudoku number: (14,)
Solved sudoku number: (15,)
Solved sudoku number: (16,)
Solved sudoku number: (17,)
Solved sudoku number: (18,)
Solved sudoku number: (19,)
Solved sudoku number: (20,)
Solved sudoku number: (21,)
Solved sudoku number: (22,)
Solved sudoku number: (23,)
Solved sudoku number: (24,)
Solved sudoku number: (25,)
Solved sudoku number: (26,)
Solved sudoku number: (27,)
Solved sudoku number: (28,)
Solved sudoku number: (29,)
Solved sudoku number: (30,)
Solved sudoku number: (31,)
Solved sudoku number: (32,)
Solved sudoku number: (33,)
Solved sudoku number: (34,)
Solved sudoku number: (35,)
So

Solved sudoku number: (287,)
Solved sudoku number: (288,)
Solved sudoku number: (289,)
Solved sudoku number: (290,)
Solved sudoku number: (291,)
Solved sudoku number: (292,)
Solved sudoku number: (293,)
Solved sudoku number: (294,)
Solved sudoku number: (295,)
Solved sudoku number: (296,)
Solved sudoku number: (297,)
Solved sudoku number: (298,)
Solved sudoku number: (299,)
Solved sudoku number: (300,)
Solved sudoku number: (301,)
Solved sudoku number: (302,)
Solved sudoku number: (303,)
Solved sudoku number: (304,)
Solved sudoku number: (305,)
Solved sudoku number: (306,)
Solved sudoku number: (307,)
Solved sudoku number: (308,)
Solved sudoku number: (309,)
Solved sudoku number: (310,)
Solved sudoku number: (311,)
Solved sudoku number: (312,)
Solved sudoku number: (313,)
Solved sudoku number: (314,)
Solved sudoku number: (315,)
Solved sudoku number: (316,)
Solved sudoku number: (317,)
Solved sudoku number: (318,)
Solved sudoku number: (319,)
Solved sudoku number: (320,)
Solved sudoku 

Solved sudoku number: (570,)
Solved sudoku number: (571,)
Solved sudoku number: (572,)
Solved sudoku number: (573,)
Solved sudoku number: (574,)
Solved sudoku number: (575,)
Solved sudoku number: (576,)
Solved sudoku number: (577,)
Solved sudoku number: (578,)
Solved sudoku number: (579,)
Solved sudoku number: (580,)
Solved sudoku number: (581,)
Solved sudoku number: (582,)
Solved sudoku number: (583,)
Solved sudoku number: (584,)
Solved sudoku number: (585,)
Solved sudoku number: (586,)
Solved sudoku number: (587,)
Solved sudoku number: (588,)
Solved sudoku number: (589,)
Solved sudoku number: (590,)
Solved sudoku number: (591,)
Solved sudoku number: (592,)
Solved sudoku number: (593,)
Solved sudoku number: (594,)
Solved sudoku number: (595,)
Solved sudoku number: (596,)
Solved sudoku number: (597,)
Solved sudoku number: (598,)
Solved sudoku number: (599,)
Solved sudoku number: (600,)
Solved sudoku number: (601,)
Solved sudoku number: (602,)
Solved sudoku number: (603,)
Solved sudoku 

Solved sudoku number: (853,)
Solved sudoku number: (854,)
Solved sudoku number: (855,)
Solved sudoku number: (856,)
Solved sudoku number: (857,)
Solved sudoku number: (858,)
Solved sudoku number: (859,)
Solved sudoku number: (860,)
Solved sudoku number: (861,)
Solved sudoku number: (862,)
Solved sudoku number: (863,)
Solved sudoku number: (864,)
Solved sudoku number: (865,)
Solved sudoku number: (866,)
Solved sudoku number: (867,)
Solved sudoku number: (868,)
Solved sudoku number: (869,)
Solved sudoku number: (870,)
Solved sudoku number: (871,)
Solved sudoku number: (872,)
Solved sudoku number: (873,)
Solved sudoku number: (874,)
Solved sudoku number: (875,)
Solved sudoku number: (876,)
Solved sudoku number: (877,)
Solved sudoku number: (878,)
Solved sudoku number: (879,)
Solved sudoku number: (880,)
Solved sudoku number: (881,)
Solved sudoku number: (882,)
Solved sudoku number: (883,)
Solved sudoku number: (884,)
Solved sudoku number: (885,)
Solved sudoku number: (886,)
Solved sudoku 

In [46]:
acc

14

In [44]:
def solve_batch (np_boards, np_solved_boards, model):
    #solved = []
    np_results = np.empty(np_boards.shape[0],dtype=np.bool_)
    for i in np.ndindex(np_boards.shape[:1]):
        #add solved board to list
        solved = solve_human_approach(np_boards[i], model)
        #add
        np_results[i] = compare_sudoku(solved, np_solved_boards[i])
        print(f"Solved sudoku number: {i}")
    correct_boards = np_results.sum()
    #accuracy = correct_boards / np_boards.shape[0]
    return correct_boards, solved

In [36]:
def solve_human_approach (np_board, model): 
    
    blanks_count = 81 - np.count_nonzero(np_board)
    np_board_1d = np_board.flatten()
    np_board_4d = np.expand_dims(to_categorical(np_board), axis=0)

    for _ in range(blanks_count):
        predictions = np.array(model.predict(np_board_4d))
        propabilities = predictions.max(2).T[0]

        #get predicted values to each cell
        values = predictions.argmax(2).T[0]
        values = values + 1                              #change range of digits from <0,8> to <1,9>

        #get position with the highest propability
        zeros = (np_board_1d == 0)
        blanks_indices = np.where(zeros)[0]
        position_to_fill = blanks_indices[propabilities[zeros].argmax()]
        #fill position with the highest propability
        value_to_fill = values[position_to_fill]

        np_board_1d[position_to_fill] = value_to_fill
        np_board = np_board_1d.reshape(9,9,1)
        np_board_4d = np_board_4d = np.expand_dims(to_categorical(np_board), axis=0)
    
    return np_board
    

In [25]:
def compare_sudoku (np_sudoku1, np_sudoku2):
    #True/False array
    comparision = np_sudoku1 == np_sudoku2
    #print(np.count_nonzero(comparision))
    return comparision.all()

In [29]:
def delete_digits(X, n_delet=1):
    """
    This function is used to create sudoku quizzes from solutions
    
    Parameters
    ----------
    X (np.array), shape (?, 9, 9, 9|10): input solutions grids.
    n_delet (integer): max number of digit to suppress from original solutions
    
    Returns
    -------
    grids: np.array of grids to guess in one-hot way. Shape: (?, 9, 9, 10)
    """
    grids = X.argmax(3)  # get the grid in a (9, 9) integer shape
    for grid in grids:
        grid.flat[np.random.randint(0, 81, n_delet)] = 0  # generate blanks (replace = True)
        
    return to_categorical(grids)

In [96]:
x = delete_digits(X_train,0)
x.shape

(75000, 9, 9, 10)

In [89]:
y =  [y_train[:, i, j, :] for i in range(9) for j in range(9)]

In [95]:
y[6].shape

(75000, 9)